In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, fixed, interact, interactive, interact_manual, Label, Layout, Textarea
import json

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 20ex !important; font-weight: bold;}
.widget-checkbox {margin-left: -60px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))

class ConfigForm(object):
    # Define widgets
    caption_msg = '<h3>Configure Your Action</h3><p>This does something.</p>'
    caption = widgets.HTML(value=caption_msg)
    button = widgets.Button(description='Submit')
    _id = widgets.Text(value='')
    path = widgets.Text(value='')
    publications = widgets.Textarea(value='', placeholder="List each publication on a separate line.")
    description = widgets.Textarea(value='')
    collectors = widgets.Textarea(value='', placeholder="List each collector on a separate line.")
    startdate = widgets.Text(value='')
    enddate = widgets.Text(value='')
    query_terms = widgets.Textarea(value='', placeholder="List each query term on a separate line.")
    processes = widgets.Textarea(value='', placeholder="List each process on a separate line.")
    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    form_items = [
        Box([Label(value='_id (required):'), _id], layout=flex),
        Box([Label(value='path (required):'), path], layout=flex),
        Box([Label(value='publications (required):'), publications], layout=flex),
        Box([Label(value='description (required):'), description], layout=flex),
        Box([Label(value='collectors (required):'), collectors], layout=flex),
        Box([Label(value='startdate (required):'), startdate], layout=flex),
        Box([Label(value='enddate (optional):'), enddate], layout=flex),
        Box([Label(value='queryterms (optional):'), query_terms], layout=flex),
        Box([Label(value='processes (optional):'), processes], layout=flex)
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self._id = ConfigForm.form_items[0]
        self.path = ConfigForm.form_items[1]
        self.publications = ConfigForm.form_items[2]
        self.description = ConfigForm.form_items[3]
        self.collectors = ConfigForm.form_items[4]
        self.startdate = ConfigForm.form_items[5]
        self.enddate = ConfigForm.form_items[6]
        self.query_terms = ConfigForm.form_items[7]
        self.processes = ConfigForm.form_items[8]

        def split_list(str):
            seq = str.split('\n')
            for key, value in enumerate(seq):
                seq[key] = value.strip()
            return seq    
                    
        def handle_submit(values):
            self.values = {}
            self.values['_id'] = ConfigForm._id.value.strip()
            self.values['path'] = ConfigForm.path.value.strip()
            self.values['publications'] = ConfigForm.publications.value.strip()
            self.values['description'] = ConfigForm.description.value.strip()
            self.values['collectors'] = ConfigForm.collectors.value.strip()
            self.values['startdate'] = ConfigForm.startdate.value.strip()
            self.values['enddate'] = ConfigForm.enddate.value.strip()
            self.values['query_terms'] = split_list(ConfigForm.query_terms.value)
            self.values['processes'] = split_list(ConfigForm.processes.value)
            required = ['_id', 'path', 'publications', 'description', 'collectors', 'startdate']
            error = False
            for item in required:
                try:
                    assert self.values[item]
                except:
                    error  = True
                    print('A value for ' + item + ' is required.')
            if error != True:
                manifest = json.dumps(self.values, indent=4)
                msg = '''You created the following manifest. To import it to the database, 
                click the "Execute Action" cell below.'''
                print(msg)
                print(manifest)

        # Initialise widgets in the container Box
        self.form = Box([self._id, self.path, self.publications, self.description, self.collectors,
                         self.startdate, self.enddate, self.query_terms, self.processes],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form

        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.button)
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)